In [1]:
root_folder = "../results"

In [2]:
metrics2paper = {
    "R2": "R2",
    "RMSE": "RMSE",
    "NRMSE": "%RMSE",
}

In [3]:
import pandas as pd
from glob import glob
from experiments_to_run import *

paths = glob(f'{root_folder}/metrics*')

df = []
for path in paths:
    tmp_df = pd.read_pickle(path)
    df.append(tmp_df)
df = pd.concat(df)

metrics = ["R2", "RMSE", "NRMSE"]
mean_df = df.groupby(["target", "config", "model"]).agg("mean")[metrics]#.reset_index()
std_df = df.groupby(["target", "config", "model"]).agg("std")[metrics]#.reset_index()

for target in mean_df.index.levels[0]:
    for config in mean_df.index.levels[1]:
        item = pd.DataFrame(mean_df.loc[target, config][metrics].mean().to_dict(), index=[(target, config, "▸ Average")])
        mean_df = pd.concat((mean_df, item))

for target in std_df.index.levels[0]:
    for config in std_df.index.levels[1]:
        item = pd.DataFrame(std_df.loc[target, config][metrics].mean().to_dict(), index=[(target, config, "▸ Average")])
        std_df = pd.concat((std_df, item))

std_df.columns = ["s_"+x for x in std_df.columns]
overall_df = pd.concat((mean_df, std_df), axis=1)

for target in TARGETS:
    tmp = overall_df.reset_index()[(overall_df.reset_index().target==target)].drop(columns=["target"])
    print("results exported to", f"{root_folder}/overall_metrics--{target}.[pickle/csv]")
    tmp.to_pickle(f"{root_folder}/overall_metrics--{target}.pickle")
    tmp.to_csv(f"{root_folder}/overall_metrics--{target}.csv", index=False)


for metric in metrics:
    overall_df[f"str_{metric}"] = overall_df.apply(lambda row: f"{row[metric]:.2f} ± {row['s_'+metric]:.2f}", axis=1)
    
overall_df = overall_df.drop(columns = [x for x in overall_df.columns if "str" not in x])
overall_df.columns = metrics
overall_df = overall_df.reset_index()

overall_df.sort_values(["target","config", "model"], inplace=True)
overall_df.rename(columns=metrics2paper, inplace=True)
overall_df

results exported to ../results/overall_metrics--CS.[pickle/csv]
results exported to ../results/overall_metrics--CSE.[pickle/csv]


/tmp/ipykernel_7033/3160420288.py:19: PerformanceWarning: indexing past lexsort depth may impact performance.
  item = pd.DataFrame(mean_df.loc[target, config][metrics].mean().to_dict(), index=[(target, config, "▸ Average")])
/tmp/ipykernel_7033/3160420288.py:19: PerformanceWarning: indexing past lexsort depth may impact performance.
  item = pd.DataFrame(mean_df.loc[target, config][metrics].mean().to_dict(), index=[(target, config, "▸ Average")])
/tmp/ipykernel_7033/3160420288.py:19: PerformanceWarning: indexing past lexsort depth may impact performance.
  item = pd.DataFrame(mean_df.loc[target, config][metrics].mean().to_dict(), index=[(target, config, "▸ Average")])
/tmp/ipykernel_7033/3160420288.py:19: PerformanceWarning: indexing past lexsort depth may impact performance.
  item = pd.DataFrame(mean_df.loc[target, config][metrics].mean().to_dict(), index=[(target, config, "▸ Average")])
/tmp/ipykernel_7033/3160420288.py:19: PerformanceWarning: indexing past lexsort depth may impact

,target,config,model,R2,RMSE,%RMSE
0,CS,Conf1,AdaBoost,0.26 ± 0.09,52.61 ± 4.59,68.58 ± 6.96
1,CS,Conf1,BaggedDT,0.30 ± 0.08,51.01 ± 4.04,66.49 ± 6.18
2,CS,Conf1,BayesianNN,0.13 ± 0.03,57.22 ± 5.97,74.60 ± 8.76
3,CS,Conf1,CatBoost,0.32 ± 0.07,50.17 ± 4.11,65.40 ± 6.39
4,CS,Conf1,GBDT,0.31 ± 0.05,50.81 ± 4.22,66.23 ± 6.46
...,...,...,...,...,...,...
92,CSE,Conf4,RF,0.42 ± 0.12,0.91 ± 0.09,53.84 ± 5.38
93,CSE,Conf4,SVR,0.03 ± 0.11,1.17 ± 0.07,69.63 ± 4.86
94,CSE,Conf4,StackEns,0.38 ± 0.15,0.93 ± 0.11,55.53 ± 6.78
95,CSE,Conf4,XGBoost,0.36 ± 0.19,0.94 ± 0.09,55.88 ± 4.55


In [4]:
for target in TARGETS:
    tmp = overall_df[(overall_df.target==target)].drop(columns=["target"])
    print("results exported to", f"{root_folder}/table_{target}.[txt/xlsx]")
    tmp.to_markdown(f"{root_folder}/table_{target}.txt", index=False, tablefmt="fancy_grid")
    tmp.to_excel(f"{root_folder}/table_{target}.xlsx", index=False)
    
    print("results exported to", f"../figures_and_tables/table_performance_{target}.[txt/xlsx/csv]")
    tmp.to_markdown(f"../figures_and_tables/table_performance_{target}.txt", index=False, tablefmt="fancy_grid")
    tmp.to_excel(f"../figures_and_tables/table_performance_{target}.xlsx", index=False)
    tmp.to_csv(f"../figures_and_tables/table_performance_{target}.csv", index=False)

results exported to ../results/table_CS.[txt/xlsx]
results exported to ../figures_and_tables/table_performance_CS.[txt/xlsx/csv]
results exported to ../results/table_CSE.[txt/xlsx]
results exported to ../figures_and_tables/table_performance_CSE.[txt/xlsx/csv]
